   # Axion-like particles analysis of the NGC1275 data

In [ ]:
### Python modules ###

%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('xtick', labelsize=20)   
plt.rc('ytick', labelsize=20)
plt.rc('text', usetex=True)
plt.rc('font', family='serif',size=25)

import numpy as np
import glob
import astropy.units as u
from pathlib import Path
from astropy.io import fits
from astropy.coordinates import SkyCoord, Angle
from regions import CircleSkyRegion, PointSkyRegion
from scipy.stats import chi2
import scipy.special as scipys
from scipy.optimize import curve_fit
from scipy.stats import gamma
import pickle

In [ ]:
### Gammapy modules ###

from gammapy.modeling import Fit
import gammapy.irf as irf
from gammapy.irf import load_cta_irfs
from gammapy.data import Observation, Observations, DataStore
from gammapy.utils.random import get_random_state
from gammapy.maps import MapAxis

# models modules
from gammapy.modeling.models import (
    Model,
    Models,
    SkyModel,
    PowerLawSpectralModel,
    PowerLawNormSpectralModel,
    ExpCutoffPowerLawSpectralModel,
    PointSpatialModel,
    GaussianSpatialModel,
    TemplateSpatialModel,
    FoVBackgroundModel,
    SpectralModel,
    #Parameter, 
    TemplateSpectralModel
)
# dataset modules
from gammapy.datasets import (
    MapDataset, 
    MapDatasetOnOff, 
    MapDatasetEventSampler,
    SpectrumDatasetOnOff,
    SpectrumDataset, 
    Datasets,
    FluxPointsDataset
)

from gammapy.maps import MapAxis, WcsGeom, Map, RegionGeom
from gammapy.makers import MapDatasetMaker, SpectrumDatasetMaker, ReflectedRegionsBackgroundMaker, WobbleRegionsFinder
from gammapy.estimators import FluxPointsEstimator

In [ ]:
### ALPs modules ###

from ALPgrid import ALPgrid, previous_limits

## Import the fits files, IACT data of the NGC1275

In [ ]:
observations_flare = Observations()
for filename in glob.glob(f"../../arqus/data/ngc1275/magic/*fits"):
    observations_flare.append(Observation.read(filename))

In [ ]:
print(observations_flare)

### Load in the spectral models that we will be using (PWL & EPWL)

In [ ]:
epwl = ExpCutoffPowerLawSpectralModel()
            
epwl.amplitude.min = 0
epwl.index.min = 0
epwl.lambda_.min = 0
epwl.reference.min = 0.3
    
model_epwl = SkyModel(spectral_model=epwl, name="EPWL")
#print(model_epwl)

In [ ]:
pwl = PowerLawSpectralModel()

pwl.amplitude.min = 0
pwl.index.min = 0
pwl.reference.min = 0.3 

model_pwl = SkyModel(spectral_model=pwl, name="PWL")
#print(model_pwl)

#### In order to load in the IACT dataset, we need to define the sky region in which the source is located, it's geometry and number of off regions to be used for estimating the background

In [ ]:
source_coordinates = SkyCoord.from_name("NGC1275")
on_center = PointSkyRegion(source_coordinates)

In [ ]:
n_off_regions = 3
region_finder = WobbleRegionsFinder(n_off_regions=n_off_regions)
bkg_maker = ReflectedRegionsBackgroundMaker(region_finder=region_finder)

## Create a dataset using our observations and definded geometry

In [ ]:
datasets_flare = Datasets()

### Define the energy axes of the dataset for producing the spectrum

In [ ]:
# ENERGY AXES
emin             = 80*u.GeV
emax             = 2.1*u.TeV
en_edges         = np.logspace(  np.log10( emin/u.GeV).value  , np.log10(emax/u.GeV).value,27) * u.GeV 
energy_axis      = MapAxis.from_edges(en_edges, interp='log' , unit="GeV", name="energy")
energy_true_axis = MapAxis.from_energy_bounds(5, 5e4, nbin=40, per_decade=False, unit="GeV", name="energy_true")

### Define and fill the dataset

In [ ]:
# EMPTY DATASET AND DATASET MAKER
geom = RegionGeom.create(region=on_center, axes=[energy_axis])
    
# spectrum dataset and its maker (fills the ON counts)
dataset_empty = SpectrumDataset.create(
    geom=geom, energy_axis_true=energy_true_axis
)
dataset_maker = SpectrumDatasetMaker(
    containment_correction=False, selection=["counts", "exposure", "edisp"]
)


for obs in observations_flare:
    dataset = dataset_maker.run(dataset_empty.copy(name=f"{obs.obs_id}"), obs)
    dataset.mask_fit = dataset.counts.geom.energy_mask(emin, emax)
    dataset_on_off = bkg_maker.run(dataset, obs)
    dataset_on_off.mask_fit = dataset_on_off.counts.geom.energy_mask(emin, emax)
    datasets_flare.append(dataset_on_off)

### Stack the observations for easier handling the data and fitting the functions

In [ ]:
datasets_flare_epwl        = datasets_flare.stack_reduce(name="NGC1275")
epwl2                 = epwl.copy()
epwl2.tag[0]          ='ExpCutoffPowerLawSpectralModel'
model                 = SkyModel(spectral_model=epwl2, name="NGC1275")
datasets_flare_epwl.models = model

### Fit the dataset to the smooth function (model that we defined) and print the results 

In [ ]:
fit           = Fit()
results_flare_epwl = fit.run(datasets_flare_epwl)

print(datasets_flare_epwl)
print(results_flare_epwl.parameters.to_table())
datasets_flare_epwl.plot_fit()

We are going to save this best fit model in order to plot it later in comparison to the best ALPs model 

In [ ]:
best_fit_model = model.copy()
model_out_dir = f""
Path(model_out_dir).mkdir(exist_ok=True, parents=True)
Models(best_fit_model).write(
    f"{model_out_dir}/best_fit.yaml", write_covariance=True, overwrite=True
)

### Extract the fluxpoints of the dataset

In [ ]:
%%time
e_min, e_max = 0.08, 2.1
energy_edges = np.geomspace(e_min, e_max, 21) * u.TeV

fpe = FluxPointsEstimator(
    energy_edges=energy_edges, source="NGC1275", selection_optional="all"
)
flux_points = fpe.run(datasets=datasets_flare_epwl)

In [ ]:
flux_points_dataset = FluxPointsDataset(
    data=flux_points, models=datasets_flare_epwl.models
)

In [ ]:
plt.figure(figsize=(14, 6))
ax = flux_points.plot(sed_type="e2dnde", color="darkorange")
flux_points.plot_ts_profiles(ax=ax, sed_type=r"e2dnde")
emin, emax = 80, 2100#5e-2,0.3e1
ymin, ymax = 4e-12,3e-10
ax.set_xlim([emin, emax])
ax.set_ylim([ymin, ymax])
ax.set_ylabel(r"$E^2 dN/dE \quad  [ erg \; cm^{-2} s^{-1}] $",size=20)
for e in dataset_on_off.counts.geom.axes["energy"].edges:
    ax.vlines( e,ymin,ymax, color="black", alpha=0.1 ) 


#### Exercise: Repeat the fit of the dataset using the power-law function, and extract the fluxpoints.

In [ ]:
%load ./Solutions/pwl_fit.py


## ALPs analysis

In [ ]:
%%time
#We define an empty list which will contain the -2logL (i.e. stat_sum() ) 
list_logL_per_gridpoint_ngc1275 = []
# for all ALP-grid points and for all magnetic field realization:
# Example:
# list_logL_per_gridpoint[k][m,g]
# is the -2logL for the k-th magnetic field realization 
# and for the ALP model with mass m and coupling g

dict_datasets = { }
path_name = "./Pyy_models/Models" 
for name in glob.glob(path_name+"/*.npy"):  

    ## GET INFO ON M AND G FROM FILE NAME
    names_split = str.split(name, "_")
    g = names_split[-2]
    m = names_split[-4]
    g = float(g) * 1e-14 / u.GeV #these are just to normalize the names of the files given by the parameters of the models
    m = float(m) * 1e-2 * u.neV

    en_absorp_array     = np.load(name)
    energy              = en_absorp_array[0] * u.GeV
    values              = en_absorp_array[1+25] * u.Unit("") # whatever numer from 1 to 100
    absorption          = TemplateSpectralModel(energy, values)
    spectral_model      = absorption * epwl
    model               = SkyModel(spectral_model=spectral_model, name="NGC1275")
    new_datasets        = datasets_flare_epwl.copy()
    new_datasets.models = model
    dict_datasets[g,m]  = new_datasets


In [ ]:
alp = ALPgrid(dict_datasets)
# we plot the grid of points we are considering
fig, ax = alp.plot_grid()
# optional, add previous limits
previous_limits(ax)

In [ ]:
%%time
# this might take some time since the function 
# get_TS_per_gridpoint() will run the fit class on all datasets
# saved in the ALPgrid class
alp.get_TS_per_gridpoint()

In [ ]:
# According to the Wilks theorem a CL of 95 or 99 % is reached for TS 6.0 and 9.2
contour_dict = dict( colors=["black"] , levels=[6.0, 9.2], linewidths=[4,3,2,1], linestyles="--" ) 
# OR WITH PREVIOUS LIMITS AS COMPARISON
fig, ax = alp.plot_grid(show='TS',contour=True,lines=True,**contour_dict)
#ax.set_ylim( [ 14.73612599e-11,5e-10] )
ax.set_ylim( [ 4.5e-13,5e-10] )
#ax.set_ylim( [ 4.5e-13,9.5e-11] )
ax.set_xlim( [ 9e-10,1.2e-6] )

In [ ]:
# you can set alp_index to the one that maximize the likelihood (best fit) ....
alp_index = np.argmin( list( alp.TS_per_gridpoint.values() ) )
# .... or to the one that minimize the likelihood (worst fit) ....
#alp_index = np.argmax( list( alp.TS_per_gridpoint.values() ) )
# ... or choose the one that you prefer
#best_index = 0

m, g = list(alp.TS_per_gridpoint.keys())[alp_index ]
print(m)
print(g)

In [ ]:
chosen_dataset  = alp.dataset_per_gridpoint[m,g]
fit             = Fit()
results         = fit.run(chosen_dataset)
chosen_dataset.stat_sum()

In [ ]:
malp = chosen_dataset.models['NGC1275']

e_min, e_max = 5e-2*u.TeV ,0.3e1*u.TeV

energies = np.geomspace(e_min, e_max, 1000) 
fluxies  = malp.spectral_model(energies)*energies**2
fluxies  = fluxies.to( u.Unit(" erg cm-2 s-1 ") )


kwargs_fp    = dict(label="DL3 flux points")
kwargs_fp.setdefault("label", "Flux points")
kwargs_fp.setdefault("sed_type", "e2dnde")

kwargs_spectrum = dict(
    kwargs_model = dict(label="Fit on DL3 flux points"),
    kwargs_fp    = dict(label="DL3 flux points")
)

ax = flux_points_dataset.plot_fit(kwargs_spectrum=kwargs_spectrum)

ax[0].plot(energies,fluxies ,c="goldenrod",label="Best ALP model")

emin, emax = 5e-2,0.3e1
ymin, ymax = 2e-12,3e-10
#ax[0].set_xlim([emin, emax])
ax[0].set_ylim([ymin, ymax])

ax[0].set_ylabel(r"$E^2 dN/dE \quad  [ erg \; cm^{-2} s^{-1}] $",size=20)
ax[1].set_ylabel(r"$Residuals$",size=20) #data-model / model
#ax.set_ylabel(r"$E^2 dN/dE \quad  [ erg \; cm^{-2} s^{-1}] $",size=10)
#for e in dataset_on_off.counts.geom.axes["energy"].edges:
#    ax.vlines( e,ymin,ymax, color="black", alpha=0.1 ) 
#ax.legend(fontsize=10)
ax[0].legend(fontsize=10)
plt.subplots_adjust(hspace=10.0)

## How to set the contrains on the ALPs paramter space? (remember the Wilks' theorem)

In [ ]:
# %load ./solutions/W_theorem.py
Write the Wilks' theorem here



#### We can load in the pre-calculated log likelihood values to check the distirbution of the test statistics and decide the way to constrain the ALPs space. In this case, we will use the TS of the null hypothesis and compare it to a TS distribution of the alternative hypothesis coming from 100 simulations of the same dataset.

#### Null-hypothesis fit

In [ ]:
path_to_folder=("./Stats/ts_list_sim_null.p")
with open(f'{path_to_folder}', 'rb') as fp:
    TS_list_sim = pickle.load(fp)

In [ ]:
# %load ./Solutions/TS_null.py
sets = list(range(0,100))
#sets = list(range(0,70))
TS_list_sim = []
for arg in sets:
    # WE SAVE THE NULL HYPOTESIS logL
    logL_null = list_logL_null[arg]
    # WE LOAD THE SIMULATION
    path_to_folder=("./Stats/List_per_simulation/list_logL_per_gridpoint_model_simulation_{}.p".format(arg+1))
    with open(f'{path_to_folder}', 'rb') as fp:
        list_logL_arg = pickle.load(fp)
        # SAVE THE TRUE VALUE OF m and g USED FOR THE SIMULATION
        mgtrue = list( list_logL_arg[0].keys() )[61]
        # FOR EACH ALP GRIDPOINT WE FIND THE 95TH PERCENTILE
        # AND WE SAVE ALL THESE VALUES IN dictionary logL_Bfield
        logL_Bfield = {}
        for mgg in list_logL_arg[0].keys():
            listLogL_perBfield = [list_logL_arg[k][mgg] for k in range(100)]
            percentile         = 5
            logL               =  np.sort( listLogL_perBfield)[percentile]
            logL_Bfield[mgg]   = logL
        # WE CONVERT THE DICT TO A LIST
        list_logL_Bfield =  [logL_Bfield[key] for key in logL_Bfield.keys()]
        # WE ADD THE NULL HYPOTESIS logL
        list_logL_Bfield.append(logL_null)
        # WE LOOK FOR THE MINIMUM VALUE
        logL_min  = np.min(list_logL_Bfield )
        # WE ARE FINALLY READY FOR COMPUTING THE TS FOR THE arg-th SIMULATION
        # IF DATA SIMULATED WITH TRUE HYPOTHESIS DO THIS
        logL_True_Hypot = logL_null
        # IF DATA SIMULATED WITH HYPOTHESIS mgtrue
        #logL_True_Hypot  =  logL_Bfield[mgtrue]
        TS  =  logL_True_Hypot -  logL_min
        TS_list_sim.append(TS)


#### Plot the distribution of the TS  from the null hypothesis

In [ ]:
from numpy import arange

#### Along with the chi^2 function, we want to plot it's generalisation, gamma function, in order to plot it with our TS distribution.

In [ ]:
def gamma_func(x, alpha, beta):
    return gamma.cdf(x, a=alpha , scale=1/beta )

In [ ]:
fig, ax = plt.subplots(figsize=(15,10),nrows=1, ncols=1)


DOF = 2 # Number of free parameters that have been fitted

min_TS      = 0 #np.min(TS_list)
max_TS      = np.max(TS_list_sim)
TS_bins     = np.linspace(min_TS ,max_TS,1500)
TS_CDF      = np.array([np.sum( TS_list_sim  < i ) for i in TS_bins])/len(TS_list_sim)
ax.plot(TS_bins, TS_CDF , color='blue', label="Null hypothesis CDF")

popt, pcov = curve_fit(gamma_func, TS_bins, TS_CDF)
fun1 = gamma_func(TS_bins, popt[0], popt[1])
ax.plot(TS_bins, fun1, alpha=0.3, linewidth=4, label=r"$\Gamma$ function")

Chi2_CDF = chi2.cdf(TS_bins,df=DOF )
ax.plot(TS_bins,Chi2_CDF, color='black',alpha=0.3, linewidth=4, label=r"CDF of $\chi^2_{df = "+str(DOF) +"}$")
ax.set_ylabel("CDF")
ax.set_xlabel("Statistic")
ax.set_title("CDF of the Statistic for the Binned case - null hypothesis")
ax.legend(loc="lower right")

#### Let's do the same for the alternative hypothesis

In [ ]:
path_to_folder=("./Stats/ts_list_sim_alt.p")
with open(f'{path_to_folder}', 'rb') as fp:
    TS_list_alt = pickle.load(fp)

In [ ]:
%load ./Solutions/TS_alt.py

#### Plot the distribution of the TS  from the alternative hypothesis, along with the chi^2 and the TS distribution of the null hypothesis.

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))#,nrows=1, ncols=1)


DOF = 2 # Number of free parameters that have been fitted

min_TS      = 0 #np.min(TS_list)
max_TS      = 30#np.max(TS_list_sim)
TS_bins     = np.linspace(min_TS ,max_TS,1500)
TS_CDF      = np.array([np.sum( TS_list_sim  < i ) for i in TS_bins])/len(TS_list_sim)
ax.plot(TS_bins, TS_CDF , color='blue', label="CDF - null hypothesis")

#Chi2_CDF = chi2.cdf(TS_bins,df=DOF )

popt, pcov = curve_fit(gamma_func, TS_bins, TS_CDF)
fun1 = gamma_func(TS_bins, popt[0], popt[1])
ax.plot(TS_bins, fun1, alpha=0.3, linewidth=4, label=r"$\Gamma$(2.72,\;0.74)")


min_TS_alt      = 0 #np.min(TS_list)
max_TS_alt     = 30#np.max(TS_list_alt)
TS_bins_alt     = np.linspace(min_TS_alt ,max_TS_alt,1500)
TS_CDF_alt      = np.array([np.sum( TS_list_alt  < i ) for i in TS_bins_alt_1])/len(TS_list_alt)
ax.plot(TS_bins_alt_1, TS_CDF_alt , color='green', label=r"CDF - alternative hypothesis ($m_a=215.44\;\mathrm{neV},g_{a\gamma}=8\times10^{-11}\mathrm{GeV}^{-1}$)")
Chi2_CDF_alt = chi2.cdf(TS_bins_alt,df=DOF )

popt_alt, pcov_alt = curve_fit(gamma_func, TS_bins_alt, TS_CDF_alt)
fun2 = gamma_func(TS_bins_alt, popt_alt[0], popt_alt[1])
ax.plot(TS_bins_alt, fun2, alpha=0.3, linewidth=4, color='green',label=r"$\Gamma$(2.45,\;0.25)")

plt.plot(TS_bins_alt,Chi2_CDF_alt, color='black',alpha=0.3, linewidth=4, label=r"CDF of $\chi^2_{df = "+str(DOF) +"}$")
plt.legend(loc="lower right", fontsize=18)

Now, let's find the effective number of d.o.f. for which the gamma function is fitting our TS distributions and giving us 95% and 99% confidence levels.

In [ ]:
%load ./Solutions/null_fit_95.py


In [ ]:
%load ./Solutions/null_fit_99.py


In [ ]:
%load ./Solutions/alt_fit_95.py

In [ ]:
%load ./Solutions/alt_fit_99.py
